In [6]:
from neurodiffeq import diff
from neurodiffeq.solvers import Solver1D, Solver2D
from neurodiffeq.conditions import IVP, DirichletBVP2D
from neurodiffeq.networks import FCNN, SinActv, Resnet, Swish, MonomialNN
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import torch
import torch.nn as nn
from neurodiffeq.generators import Generator1D
import math
from neurodiffeq.monitors import Monitor1D
import IPython
import numpy as np
import matplotlib.pyplot as plt

   
from scipy.integrate import odeint

In [7]:
alpha_0 = 2
alpha_1 = 1
c2 = alpha_0/(12*alpha_1)
c3 = 1.0
c2
import math
def truncate(number, digits) -> float:
    # Improve accuracy with floating point operations, to avoid truncate(16.4, 2) = 16.39 or truncate(-1.13, 2) = -1.12
    nbDecimals = len(str(number).split('.')[1]) 
    if nbDecimals <= digits:
        return number
    stepper = 10.0 ** digits
    return math.trunc(stepper * number) / stepper

In [8]:
def ode_system(A, t):
    return [(1.5*alpha_1*torch.pow(t,2)*diff(A,t,order=2)-3*alpha_1*(A-1)+0.5*torch.pow(t,2)*alpha_0)*(t*diff(A,t)+2*A)-2*A*torch.pow(t,2)*alpha_0]

N = 300000
t_i = 1.0
j = 0
full = []
errors = []
while(t_i < 5.1):
    t_start = t_i
    t_end = t_i+0.1
    t = np.linspace(t_start, t_end, N)
    
    conditions = [IVP(t_0=1.0, u_0=1.0+c2-c3, u_0_prime=2*c2+2*c3)]
    nets = [Resnet(actv=Swish)]
    solver = Solver1D(ode_system, conditions, t_min=t_start, t_max=t_end, nets = nets)
    solver.fit(max_epochs=N)
    solution = solver.get_solution()
    
    A_nn = solution(t, to_numpy=True)
        
    # Initial conditions
    
    alpha_0 = 2.0
    alpha_1 = 1.0
    c2 = alpha_0/(12*alpha_1)
    c3 = 1.0
    y0  = 1.0+c2-c3
    u0  = 2*c2+2*c3
    h   = t[1]-t[0]
    
    # Enter the definition of the set of ordinary differential equations
    def ode(t,y,u):
      ode_1 = u
      ode_2 = (4*y*alpha_0)/(3*alpha_1*(t*u+2*y))  - alpha_0/(3*alpha_1) + (2*(y-1))/(t**2)
      return np.array([ode_1,ode_2])
    
    # RK4
    y = np.empty(N); u = np.empty(N)
    
    # t[0] = t_initial
    y[0] = y0; u[0] = u0
    
    for i in range(0,N-1,1):
    
      k11 = h*ode(t[i],y[i],u[i])[0]
      k12 = h*ode(t[i],y[i],u[i])[1]
    
      k21 = h*ode(t[i]+(h/2),y[i]+(k11/2),u[i]+(k12/2))[0]
      k22 = h*ode(t[i]+(h/2),y[i]+(k11/2),u[i]+(k12/2))[1]
    
      k31 = h*ode(t[i]+(h/2),y[i]+(k21/2),u[i]+(k22/2))[0]
      k32 = h*ode(t[i]+(h/2),y[i]+(k21/2),u[i]+(k22/2))[1]
    
      k41 = h*ode(t[i]+h,y[i]+k31,u[i]+k32)[0]
      k42 = h*ode(t[i]+h,y[i]+k31,u[i]+k32)[1]
    
      y[i+1] = y[i] + ((k11+2*k21+2*k31+k41)/6)
      u[i+1] = u[i] + ((k12+2*k22+2*k32+k42)/6)
  
    lambda_error = np.sqrt(np.mean((A_nn-y)**2))
    A_n5 = A_nn*(t**3)
    poly = np.polyfit(t, A_n5, deg=5)
    full.append(poly)
    errors.append(lambda_error)

    j += 1
    t_i += 0.1
    # print(poly)

Training Progress:   0%|                             | 0/300000 [00:00<?, ?it/s]

Training Progress:   0%|                             | 0/300000 [00:00<?, ?it/s]

Training Progress:   0%|                             | 0/300000 [00:00<?, ?it/s]

Training Progress:   0%|                             | 0/300000 [00:00<?, ?it/s]

Training Progress:   0%|                             | 0/300000 [00:00<?, ?it/s]

...

/var/folders/1d/4ydxxx616sj_j5v722pzy2wr0000gn/T/ipykernel_26056/598303680.py:63: RankWarning: Polyfit may be poorly conditioned
  poly = np.polyfit(t, A_n5, deg=5)


Training Progress:   0%|                             | 0/300000 [00:00<?, ?it/s]

/var/folders/1d/4ydxxx616sj_j5v722pzy2wr0000gn/T/ipykernel_26056/598303680.py:63: RankWarning: Polyfit may be poorly conditioned
  poly = np.polyfit(t, A_n5, deg=5)


Training Progress:   0%|                             | 0/300000 [00:00<?, ?it/s]

/var/folders/1d/4ydxxx616sj_j5v722pzy2wr0000gn/T/ipykernel_26056/598303680.py:63: RankWarning: Polyfit may be poorly conditioned
  poly = np.polyfit(t, A_n5, deg=5)


In [9]:
with open('full_1.txt', 'w') as f:
    for line in full:
        f.write(f"{line}\n")

In [10]:
with open('errors_1.txt', 'w') as f:
    for line in errors:
        f.write(f"{line}\n")